In [1]:
!pip install -U transformers --quiet
!pip install -q pillow accelerate einops --quiet

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer #AutoModelCausalLM for generation task
from PIL import Image

In [3]:
torch.set_default_device("cuda") #setting the default device to cuda

In [4]:
#setting up the model
model = AutoModelForCausalLM.from_pretrained(
    "MILVLG/imp-v1-3b", #this is the model imported from hugging face
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

#setting up the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "MILVLG/imp-v1-3b",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

configuration_imp.py:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/MILVLG/imp-v1-3b:
- configuration_imp.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_imp.py:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

vision_encoder.py:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
text = """A chat between a curious user and an artificial intelligence assistant.
The assistant generates helpful and detailed answer after analyzing an image.
USER: <image>\nAnalyze the image. ASSISTANT:"""
image = Image.open("/kaggle/input/kidney-image/Cross-section-kidney-blood-vessels.jpg")

In [8]:
print(model) #architecture of model

ImpForCausalLM(
  (transformer): ImpModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )
    )
    (visi

### The provided code performs text and image generation using a language model. Here's a breakdown:

- Tokenization: It tokenizes the input text using the tokenizer and converts it into PyTorch tensors (return_tensors="pt").
- Image Preprocessing: The code preprocesses the input image using the model.image_preprocess function, resulting in a tensor (torch array) representing the image.
- Text Generation: The model.generate function generates text based on the tokenized input (input_ids). It limits the output to a maximum of 800 new tokens (max_new_tokens=800), incorporating information from the provided image. The use_cache=True parameter suggests the use of cached information during generation.
- Decoding and Printing: The generated text (output_ids) is decoded using the tokenizer, skipping special tokens. The decoded text is then printed after stripping leading and trailing whitespaces.

In [10]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
image_tensor = model.image_preprocess(image) #the tensor (torch array) of the image
output_ids = model.generate(
    input_ids,
    max_new_tokens=6000,
    images=image_tensor,
    use_cache=True)[0]
print(tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip())

The image is a detailed drawing of the human kidney, showcasing its various parts and structures. The drawing is color-coded, with red, blue, and white colors representing different components of the kidney. The drawing is labeled with the name "Kidney" and provides a comprehensive view of the organ's anatomy.
